# GraphRAG

In [ ]:
import os

# ---------- LlamaIndex (índices) ----------
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    Settings,
)
from llama_index.core.indices.knowledge_graph import KnowledgeGraphIndex
from llama_index.core.indices.list.base import SummaryIndex
from llama_index.core.composability import ComposableGraph
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.llms.openai import OpenAI as LI_OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# ---------- LangGraph / LangChain (agente) ----------
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_community.document_loaders import PyMuPDFLoader

In [ ]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
NEO4J_URL = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
PDF_DIR = "../data/docs"

# Modelos OpenAI (ajusta si quieres)
# LlamaIndex (para extracción KG/embeddings) puede usar modelos “baratos”
LI_LLM_MODEL = "gpt-4o-mini"          # para LlamaIndex internamente
EMBED_MODEL = "text-embedding-3-small" # embeddings
# El agente (LangGraph) puede usar un modelo distinto si quieres
AGENT_LLM_MODEL = "gpt-4o-mini"

In [ ]:
def build_indices():
    # ---------- Config global de LlamaIndex ----------
    Settings.llm = LI_OpenAI(model=LI_LLM_MODEL, temperature=0)
    Settings.embed_model = OpenAIEmbedding(model=EMBED_MODEL)

    # ---------- 1) PDFs → documentos ----------
    docs = SimpleDirectoryReader(input_dir=PDF_DIR).load_data()
    if not docs:
        raise RuntimeError(f"No se encontraron PDFs en: {PDF_DIR}")

    # ---------- 2) Grafo en Neo4j ----------
    graph_store = Neo4jGraphStore(
        url=NEO4J_URL, username=NEO4J_USER, password=NEO4J_PASSWORD
    )

    # ---------- 3) Construir KG (tripletas → Neo4j) ----------
    kg_index = KnowledgeGraphIndex.from_documents(
        docs,
        graph_store=graph_store,
        max_triplets_per_chunk=3,
    )

    # ---------- 4) Vector index ----------
    vec_index = VectorStoreIndex.from_documents(docs)

    # ---------- 5) Composición híbrida ----------
    composed = ComposableGraph.from_indices(
        root_index_cls=SummaryIndex,
        children_indices=[kg_index, vec_index],
        index_summaries=[
            "Knowledge Graph: entidades y relaciones del corpus; útil para preguntas relacionales y de conexiones.",
            "Vector Index: pasajes textuales exactos del PDF; útil para definiciones, fórmulas y citas literales."
        ],
    )

    return kg_index, vec_index, composed


def build_tools(kg_index, vec_index, composed):
    # Motores de consulta independientes
    kg_engine = kg_index.as_query_engine()                 # centrado en grafo (entidades/relaciones)
    vec_engine = vec_index.as_query_engine(similarity_top_k=4)  # centrado en pasajes
    cmp_engine = composed.as_query_engine()                # híbrido (usa la jerarquía root+children)

    @tool
    def graph_search(query: str) -> str:
        """Devuelve hechos/relaciones extraídos desde el grafo (Neo4j)."""
        print("Graph search query:", query)
        result = str(kg_engine.query(query))
        print("Graph search result:", result)
        return result

    @tool
    def vector_search(query: str) -> str:
        """Devuelve pasajes textuales relevantes desde el índice vectorial."""
        print("Vector search query:", query)
        result = str(vec_engine.query(query))
        print("Vector search result:", result)
        return result

    @tool
    def hybrid_search(query: str) -> str:
        """Combina señales de grafo y texto para un contexto más completo."""
        print("Hybrid search query:", query)
        result = str(cmp_engine.query(query))
        print("Hybrid search result:", result)
        return result

    return [graph_search, vector_search, hybrid_search]


def build_agent(tools):
    # LLM del agente (LangGraph / LangChain)
    llm = ChatOpenAI(model=AGENT_LLM_MODEL, temperature=0)

    system_prompt = (
        "Eres un agente GraphRAG. Decide entre graph_search (para entidades/relaciones), "
        "vector_search (para pasajes textuales) o hybrid_search (para combinar). "
        "Responde citando hechos claros y di si falta contexto."
    )

    agent = create_react_agent(
        model=llm,
        tools=tools,
        prompt=system_prompt,
    )
    return agent

In [ ]:
kg_index, vec_index, composed = build_indices()
# Registrar tools
tools = build_tools(kg_index, vec_index, composed)
# Crear agente
agent = build_agent(tools)

In [ ]:
agent

In [ ]:
# Ejemplo de pregunta:
questions = [
    "¿Qué problema resuelve el mecanismo de Multi-Head Attention y por qué es útil?",
    "¿Cómo se calcula el scaled dot-product attention?",
    "¿Qué ventajas ofrece el Transformer frente a modelos recurrentes en el paper? Usa un vector search para informarte.",
    "¿Qué ventajas ofrece el Transformer frente a modelos recurrentes en el paper? Usa un graph search para informarte.",
    "¿Qué ventajas ofrece el Transformer frente a modelos recurrentes en el paper? Usa un hybrid search para informarte.",
]
for question in questions:
    print("\n" + "=" * 80)
    print("Q:", question)
    result = agent.invoke({"messages": [("user", question)]})
    print("A:", [result["messages"][-1].content] if result["messages"] else "<sin respuesta>")
